In [29]:
import pandas as pd
import numpy as np
from IPython.display import display
import dataframe_image as dfi
from typing import Literal
import seaborn as sns
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score

In [91]:
res_dict = {}
acc_dict = {}
hetero_dict = {}

base_acc_path = "/home/FL-heterogeneity/results"
base_hetero_path = "/home/FL-heterogeneity/final-het-results"
hetero_metric = "hellinger_distance"
acc_cols = {
    "IidPartitioner": ["num_partitions"],
    "DirichletPartitioner": ["num_partitions", "alpha"],
    "PathologicalPartitioner": ["num_partitions", "num_classes_per_partition"],
}


def read_data(
    ds_name,
    partitioner_name,
    base_acc_path=base_acc_path,
    base_hetero_path=base_hetero_path,
    mode: Literal["single-seed", "multi-seed", "mean"] = "multi-seed",
):
    # Read hetero
    path = f"{base_hetero_path}/{ds_name}/{partitioner_name}/compute_{hetero_metric}.csv" if ds_name != "fashion_mnist" else f"{base_hetero_path}/zalando-datasets/{ds_name}/{partitioner_name}/compute_{hetero_metric}.csv"
    metrics = pd.read_csv(path)
    metrics = metrics.drop_duplicates()

    if mode == "single-seed":
        hetero = metrics.loc[metrics["fds_seed"] == 42]
    elif mode == "multi-seed":
        hetero = metrics
    elif mode == "mean":
        hetero = metrics.groupby(acc_cols[partitioner_name])["metric_value"].mean().to_frame()

    try:
        hetero = hetero[acc_cols[partitioner_name] + ["metric_value"] + ["seed"] if mode == "multi-seed" else []]
    except:
        hetero = hetero[["metric_value"]]

    path_acc = (
        f"{base_acc_path}"
        + f"final-{ds_name}/{ds_name}/{partitioner_name}/Adam"
        + "{}"
        + "/test_res.csv"
    ) if ds_name != "fashion_mnist" else f"{base_acc_path}/final-fashion-mnist/zalando-datasets/fashion_mnist/{partitioner_name}/Adam" + "{}" +"/test_res.csv"
    acc = pd.read_csv(path_acc)
    if partitioner_name == "DirichletPartitioner":
        acc = acc[acc["self_balancing"] == False]
    if partitioner_name == "PathologicalPartitioner":
        acc = acc[acc["class_assignment_mode"] == "first-deterministic"]
    
    acc = acc[acc_cols[partitioner_name] + ["eval/acc"]]

    # Merge results
    res = acc.merge(hetero, on=acc_cols[partitioner_name])
    return res, acc, hetero

    

In [92]:
def read_acc_data(ds_name, partitioner_name, base_acc_path=base_acc_path):
    # base_acc_path = base_acc_path if ds_name in ["cifar10", "mnist"] else "/home/FL-heterogeneity/results/final-cifar100/"
    path_acc = (
        f"{base_acc_path}/"
        + f"final-{ds_name}/{ds_name}/{partitioner_name}/Adam"
        + "{}"
        + "/test_res.csv"
    ) if ds_name != "fashion_mnist" else f"{base_acc_path}/final-fashion-mnist/zalando-datasets/fashion_mnist/{partitioner_name}/Adam" + "{}" +"/test_res.csv"
    acc = pd.read_csv(path_acc)
    if partitioner_name == "DirichletPartitioner":
        acc = acc[acc["self_balancing"] == False]
    if partitioner_name == "PathologicalPartitioner":
        acc = acc[acc["class_assignment_mode"] == "first-deterministic"]
    
    acc = acc[acc_cols[partitioner_name] + ["eval/acc"] + ["fds_seed"]]
    return acc

def read_hetero_data(ds_name, partitioner_name, base_hetero_path=base_hetero_path, mode: Literal["single-seed", "multi-seed", "mean"] = "multi-seed"):
    path = f"{base_hetero_path}/{ds_name}/{partitioner_name}/compute_{hetero_metric}.csv" if ds_name != "fashion_mnist" else f"{base_hetero_path}/zalando-datasets/{ds_name}/{partitioner_name}/compute_{hetero_metric}.csv"
    metrics = pd.read_csv(path)
    metrics = metrics.drop_duplicates()

    if mode == "single-seed":
        hetero = metrics.loc[metrics["fds_seed"] == 42]
    elif mode == "multi-seed":
        hetero = metrics
    elif mode == "mean":
        hetero = metrics.groupby(acc_cols[partitioner_name])["metric_value"].mean().to_frame()

    if partitioner_name == "DirichletPartitioner":
        hetero = hetero[hetero["self_balancing"] == False]
    if partitioner_name == "PathologicalPartitioner":
        hetero = hetero[hetero["class_assignment_mode"] == "first-deterministic"]
    try:
        hetero = hetero[acc_cols[partitioner_name] + ["metric_value"] + ["fds_seed"] if mode == "multi-seed" else []]
    except:
        hetero = hetero[["metric_value"]]
    
    return hetero

def merge_data(acc, hetero, partitioner_name):
    res = acc.merge(hetero, on=acc_cols[partitioner_name] + ["fds_seed"])
    return res


ds_name = "cifar10"
partitioner_name = "IidPartitioner"
acc = read_acc_data(ds_name, partitioner_name)
hetero = read_hetero_data(ds_name, partitioner_name)
res = merge_data(acc, hetero, partitioner_name)

In [93]:
res_dict = {}
res_dict_train = {}
res_dict_test = {}
acc_dict = {}
hetero_dict = {}

for partitioner_name in ["IidPartitioner", "DirichletPartitioner", "PathologicalPartitioner"]:
    for ds_name in ["mnist", "cifar10", "cifar100", "fashion_mnist"]:
        print(ds_name, partitioner_name)
        acc_dict[(ds_name, partitioner_name)] = read_acc_data(ds_name, partitioner_name)
        hetero_dict[(ds_name, partitioner_name)] =  read_hetero_data(ds_name, partitioner_name)
        res_dict[(ds_name, partitioner_name)] = merge_data(
            acc_dict[(ds_name, partitioner_name)], 
            hetero_dict[(ds_name, partitioner_name)], 
            partitioner_name
            )
        test_mask = res_dict[(ds_name, partitioner_name)]["fds_seed"] == 46
        res_dict_train[(ds_name, partitioner_name)] = res_dict[(ds_name, partitioner_name)][~test_mask]
        res_dict_test[(ds_name, partitioner_name)] = res_dict[(ds_name, partitioner_name)][test_mask]

mnist IidPartitioner
cifar10 IidPartitioner
cifar100 IidPartitioner
fashion_mnist IidPartitioner
mnist DirichletPartitioner
cifar10 DirichletPartitioner
cifar100 DirichletPartitioner
fashion_mnist DirichletPartitioner
mnist PathologicalPartitioner
cifar10 PathologicalPartitioner
cifar100 PathologicalPartitioner
fashion_mnist PathologicalPartitioner


## All data together

In [94]:
from sklearn.metrics import mean_squared_error
import numpy as np

res_train_merged = pd.concat([df[['metric_value', 'eval/acc']] for df in res_dict_train.values()])
res_test_merged = pd.concat([df[['metric_value', 'eval/acc']] for df in res_dict_test.values()])
res_train_merged = res_train_merged.dropna()
res_test_merged = res_test_merged.dropna()


# Prepare the data for linear regression
X_train = res_train_merged['metric_value']
y_train = res_train_merged['eval/acc']
X_test = res_test_merged['metric_value']
y_test = res_test_merged['eval/acc']

# Add a constant to the independent variable
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

# Fit the linear regression model
model = sm.OLS(y_train, X_train).fit()

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Print the summary of the model and RMSE
print(model.summary())
print(f"RMSE: {rmse}")
# Calculate R-squared
r_squared = model.rsquared

# Print R-squared
print(f"R-squared: {r_squared}")

                            OLS Regression Results                            
Dep. Variable:               eval/acc   R-squared:                       0.165
Model:                            OLS   Adj. R-squared:                  0.165
Method:                 Least Squares   F-statistic:                     309.4
Date:                Sun, 15 Dec 2024   Prob (F-statistic):           2.48e-63
Time:                        20:34:29   Log-Likelihood:                -385.69
No. Observations:                1564   AIC:                             775.4
Df Residuals:                    1562   BIC:                             786.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.7674      0.015     51.550   

## Per dataset Linear Regression

In [95]:
results = {}

for ds_name in [ "cifar10", "cifar100"]:
    # Merge train and test data
    res_train_list = []
    res_test_list = []
    for partitioner_name in ["IidPartitioner", "DirichletPartitioner", "PathologicalPartitioner"]:
        res_train = res_dict_train[(ds_name, partitioner_name)]
        res_test = res_dict_test[(ds_name, partitioner_name)]
        res_train_list.append(res_train[['metric_value', 'eval/acc']])
        res_test_list.append(res_test[['metric_value', 'eval/acc']])
        
    res_train_merged = pd.concat(res_train_list)
    res_test_merged = pd.concat(res_test_list)
    res_train_merged = res_train_merged.dropna()
    res_test_merged = res_test_merged.dropna()

    # Prepare the data for linear regression
    X_train = res_train_merged['metric_value']
    y_train = res_train_merged['eval/acc']
    X_test = res_test_merged['metric_value']
    y_test = res_test_merged['eval/acc']

    # Add a constant to the independent variable
    X_train = sm.add_constant(X_train)
    X_test = sm.add_constant(X_test)

    # Fit the linear regression model
    model = sm.OLS(y_train, X_train).fit()

    # Make predictions on the train and test sets
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Calculate MAE, RMSE, and R-squared for train set
    mae_train = np.mean(np.abs(y_train - y_train_pred))
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    r_squared_train = r2_score(y_train, y_train_pred)

    # Calculate MAE, RMSE, and R-squared for test set
    mae_test = np.mean(np.abs(y_test - y_test_pred))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    r_squared_test = r2_score(y_test, y_test_pred)

    # Store the results in a nested dictionary
    results[ds_name] = {
        "train": {
            "model_summary": model.summary(),
            "mae": mae_train,
            "rmse": rmse_train,
            "r_squared": r_squared_train
        },
        "test": {
            "mae": mae_test,
            "rmse": rmse_test,
            "r_squared": r_squared_test
        }
    }

In [96]:
# Create a DataFrame from the results dictionary with multi-index columns
results_df = pd.DataFrame({
    ('Train', 'R-squared'): [round(results[ds]['train']['r_squared'], 3) for ds in results],
    ('Train', 'RMSE'): [round(results[ds]['train']['rmse'], 3) for ds in results],
    ('Train', 'MAE'): [round(results[ds]['train']['mae'], 3) for ds in results],
    ('Test', 'R-squared'): [round(results[ds]['test']['r_squared'], 3) for ds in results],
    ('Test', 'RMSE'): [round(results[ds]['test']['rmse'], 3) for ds in results],
    ('Test', 'MAE'): [round(results[ds]['test']['mae'], 3) for ds in results]
}, index=results.keys())

# Display the DataFrame
display(results_df)


Train                    Test              
         R-squared   RMSE    MAE R-squared   RMSE    MAE
cifar10      0.666  0.079  0.064     0.619  0.081  0.067
cifar100     0.684  0.040  0.033     0.710  0.038  0.033

In [97]:
# Create a LaTeX table from the results DataFrame
latex_table = results_df.to_latex(index=True, multirow=True, float_format="%.3f", caption="Linear Regression Results", label="tab:linear_regression_results").replace("\\begin{table}", "\\begin{table}\n\\begin{center}").replace("\\end{table}", "\\end{center}\n\\end{table}").replace("\\toprule", "\\hline").replace("\\midrule", "\\hline").replace("\\bottomrule", "\\hline")

# Display the LaTeX table
print(latex_table)

\begin{table}
\begin{center}
\caption{Linear Regression Results}
\label{tab:linear_regression_results}
\begin{tabular}{lrrrrrr}
\hline
 & \multicolumn{3}{r}{Train} & \multicolumn{3}{r}{Test} \\
 & R-squared & RMSE & MAE & R-squared & RMSE & MAE \\
\hline
cifar10 & 0.666 & 0.079 & 0.064 & 0.619 & 0.081 & 0.067 \\
cifar100 & 0.684 & 0.040 & 0.033 & 0.710 & 0.038 & 0.033 \\
\hline
\end{tabular}
\end{center}
\end{table}



In [98]:
# ds_name = "mnist"
# results_dict = results[ds_name]
# # Print the summary of the model, RMSE, and R-squared for each dataset
# print(f"Dataset: {ds_name}")
# print(results_dict["model_summary"])
# print(f"RMSE: {rmse}")
# print(f"R-squared: {r_squared}")
# print("\n")

In [99]:
results

{'cifar10': {'train': {'model_summary': <class 'statsmodels.iolib.summary.Summary'>
   """
                               OLS Regression Results                            
   Dep. Variable:               eval/acc   R-squared:                       0.666
   Model:                            OLS   Adj. R-squared:                  0.666
   Method:                 Least Squares   F-statistic:                     787.3
   Date:                Sun, 15 Dec 2024   Prob (F-statistic):           5.63e-96
   Time:                        20:34:30   Log-Likelihood:                 444.81
   No. Observations:                 396   AIC:                            -885.6
   Df Residuals:                     394   BIC:                            -877.7
   Df Model:                           1                                         
   Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
   --

In [100]:
ds_name = "cifar10"
results_dict = results["cifar10"]
# Print the summary of the model, RMSE, and R-squared for each dataset
print(f"Dataset: {ds_name}")
print(results_dict["train"]["model_summary"])
print(f"RMSE: {rmse}")
print(f"R-squared: {r_squared}")
print("\n")

Dataset: cifar10
                            OLS Regression Results                            
Dep. Variable:               eval/acc   R-squared:                       0.666
Model:                            OLS   Adj. R-squared:                  0.666
Method:                 Least Squares   F-statistic:                     787.3
Date:                Sun, 15 Dec 2024   Prob (F-statistic):           5.63e-96
Time:                        20:34:30   Log-Likelihood:                 444.81
No. Observations:                 396   AIC:                            -885.6
Df Residuals:                     394   BIC:                            -877.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.5806      0.

In [101]:
results_dict = results["cifar100"]
# Print the summary of the model, RMSE, and R-squared for each dataset
print(f"Dataset: {ds_name}")
print(results_dict["train"]["model_summary"])
print(f"RMSE: {rmse}")
print(f"R-squared: {r_squared}")
print("\n")

Dataset: cifar10
                            OLS Regression Results                            
Dep. Variable:               eval/acc   R-squared:                       0.684
Model:                            OLS   Adj. R-squared:                  0.683
Method:                 Least Squares   F-statistic:                     810.6
Date:                Sun, 15 Dec 2024   Prob (F-statistic):           1.17e-95
Time:                        20:34:30   Log-Likelihood:                 680.02
No. Observations:                 376   AIC:                            -1356.
Df Residuals:                     374   BIC:                            -1348.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.2390      0.

## Train on one single partitioner data and test on the subset 

In [102]:

from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Function to perform linear regression
def perform_linear_regression(X_train, y_train):
    # Add a constant to the independent variable
    X_train = sm.add_constant(X_train)

    # Fit the linear regression model
    model = sm.OLS(y_train, X_train).fit()

    return model

for ds_name in ["cifar10", "cifar100", "mnist", "fashion_mnist"]:
    print(f"{ds_name}")
    # Train on res_train from a single partitioner
    for partitioner_name in ["IidPartitioner", "DirichletPartitioner", "PathologicalPartitioner"]:
        print(f"Training model on {partitioner_name}")

        # partitioner_name = "IidPartitioner"

        res_train = res_dict_train[(ds_name, partitioner_name)].dropna()
        res_test = res_dict_test[(ds_name, partitioner_name)].dropna()

        # Prepare the training data
        X_train = res_train['metric_value']
        y_train = res_train['eval/acc']

        # Train the model
        model = perform_linear_regression(X_train, y_train)

        # Prepare the test data for the remaining partitioners
        other_partitioners = [partitioner for partitioner in ["IidPartitioner", "DirichletPartitioner", "PathologicalPartitioner"] if partitioner != partitioner_name]
        for test_partitioner_name in other_partitioners:
            res_test_merged = res_dict[(ds_name, test_partitioner_name)]
            res_test_merged = res_test_merged.dropna()

            X_test_full = res_test_merged['metric_value']
            y_test_full = res_test_merged['eval/acc']

            # Add a constant to the independent variable
            X_test_full = sm.add_constant(X_test_full)

            # Make predictions on the test set
            y_pred_full = model.predict(X_test_full)

            # Calculate RMSE
            rmse_full = np.sqrt(mean_squared_error(y_test_full, y_pred_full))

            # Calculate R-squared using scikit-learn's r2_score function
            r_squared_full = r2_score(y_test_full, y_pred_full)

            # Print the results
            print(f"Results on the full test set of partitioner {test_partitioner_name}:")
            print(f"RMSE: {rmse_full}")
            print(f"R-squared: {r_squared_full}")

        # Prepare the test data for the same partitioner
        X_test_partitioner = res_test['metric_value']
        y_test_partitioner = res_test['eval/acc']

        # Add a constant to the independent variable
        X_test_partitioner = sm.add_constant(X_test_partitioner)

        # Make predictions on the test set of the same partitioner
        y_pred_partitioner = model.predict(X_test_partitioner)

        # Calculate RMSE
        rmse_partitioner = np.sqrt(mean_squared_error(y_test_partitioner, y_pred_partitioner))

        # Calculate R-squared using scikit-learn's r2_score function
        r_squared_partitioner = r2_score(y_test_partitioner, y_pred_partitioner)

        # Print the results
        print("Results on the test set of the same partitioner:")
        print(f"RMSE: {rmse_partitioner}")
        print(f"R-squared: {r_squared_partitioner}\n")


cifar10
Training model on IidPartitioner
Results on the full test set of partitioner DirichletPartitioner:
RMSE: 0.16641476867540833
R-squared: -3.0042079721410904
Results on the full test set of partitioner PathologicalPartitioner:
RMSE: 0.22868848846587728
R-squared: -1.9640218246209176
Results on the test set of the same partitioner:
RMSE: 0.03354947457083256
R-squared: 0.6426077300424966

Training model on DirichletPartitioner
Results on the full test set of partitioner IidPartitioner:
RMSE: 0.049899744670284796
R-squared: 0.2779376244540628
Results on the full test set of partitioner PathologicalPartitioner:
RMSE: 0.11753661225896946
R-squared: 0.21704080820088678
Results on the test set of the same partitioner:
RMSE: 0.06547107631399231
R-squared: 0.35969221288323805

Training model on PathologicalPartitioner
Results on the full test set of partitioner IidPartitioner:
RMSE: 0.09893855216105346
R-squared: -1.83863249807369
Results on the full test set of partitioner DirichletParti

In [103]:
# Store the results in a DataFrame
experiment_results = []

for ds_name in ["cifar10", "cifar100", "mnist", "fashion_mnist"]:
    for partitioner_name in ["IidPartitioner", "DirichletPartitioner", "PathologicalPartitioner"]:
        # Prepare the training data
        res_train = res_dict_train[(ds_name, partitioner_name)].dropna()
        X_train = res_train['metric_value']
        y_train = res_train['eval/acc']

        # Train the model
        model = perform_linear_regression(X_train, y_train)

        # Make predictions on the train set
        y_train_pred = model.predict(sm.add_constant(X_train))

        # Calculate RMSE, R-squared, and MAE for the train set
        rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
        r_squared_train = r2_score(y_train, y_train_pred)
        mae_train = np.mean(np.abs(y_train - y_train_pred))

        # Store the train results
        experiment_results.append({
            "dataset": ds_name,
            "train_partitioner": partitioner_name,
            "test_partitioner": partitioner_name,
            "rmse": rmse_train,
            "r_squared": r_squared_train,
            "mae": mae_train,
            "set": "train"
        })

        # Prepare the test data for the remaining partitioners
        other_partitioners = [partitioner for partitioner in ["IidPartitioner", "DirichletPartitioner", "PathologicalPartitioner"] if partitioner != partitioner_name]
        for test_partitioner_name in other_partitioners:
            res_test_merged = res_dict[(ds_name, test_partitioner_name)].dropna()
            X_test_full = res_test_merged['metric_value']
            y_test_full = res_test_merged['eval/acc']

            # Add a constant to the independent variable
            X_test_full = sm.add_constant(X_test_full)

            # Make predictions on the test set
            y_pred_full = model.predict(X_test_full)

            # Calculate RMSE, R-squared, and MAE
            rmse_full = np.sqrt(mean_squared_error(y_test_full, y_pred_full))
            r_squared_full = r2_score(y_test_full, y_pred_full)
            mae_full = np.mean(np.abs(y_test_full - y_pred_full))

            # Store the results
            experiment_results.append({
                "dataset": ds_name,
                "train_partitioner": partitioner_name,
                "test_partitioner": test_partitioner_name,
                "rmse": rmse_full,
                "r_squared": r_squared_full,
                "mae": mae_full,
                "set": "test"
            })

        # Prepare the test data for the same partitioner
        res_test = res_dict_test[(ds_name, partitioner_name)].dropna()
        X_test_partitioner = res_test['metric_value']
        y_test_partitioner = res_test['eval/acc']

        # Add a constant to the independent variable
        X_test_partitioner = sm.add_constant(X_test_partitioner)

        # Make predictions on the test set of the same partitioner
        y_pred_partitioner = model.predict(X_test_partitioner)

        # Calculate RMSE, R-squared, and MAE
        rmse_partitioner = np.sqrt(mean_squared_error(y_test_partitioner, y_pred_partitioner))
        r_squared_partitioner = r2_score(y_test_partitioner, y_pred_partitioner)
        mae_partitioner = np.mean(np.abs(y_test_partitioner - y_pred_partitioner))

        # Store the results
        experiment_results.append({
            "dataset": ds_name,
            "train_partitioner": partitioner_name,
            "test_partitioner": partitioner_name,
            "rmse": rmse_partitioner,
            "r_squared": r_squared_partitioner,
            "mae": mae_partitioner,
            "set": "test"
        })

# Convert the results to a DataFrame
experiment_results_df = pd.DataFrame(experiment_results)
# Display just the test sets
# display(experiment_results_df[experiment_results_df['set'] == 'test'].iloc[:, :-1])

# Shorten the partitioner names
experiment_results_df['train_partitioner'] = experiment_results_df['train_partitioner'].replace({
    'IidPartitioner': 'iid',
    'DirichletPartitioner': 'dirichlet',
    'PathologicalPartitioner': 'pathological'
})

experiment_results_df['test_partitioner'] = experiment_results_df['test_partitioner'].replace({
    'IidPartitioner': 'iid',
    'DirichletPartitioner': 'dirichlet',
    'PathologicalPartitioner': 'pathological'
})

# Display the updated DataFrame
display(experiment_results_df[experiment_results_df['set'] == 'test'].iloc[:, :-1])
# Create a LaTeX table from the experiment results DataFrame
latex_table = experiment_results_df[experiment_results_df['set'] == 'test'].iloc[:, :-1].to_latex(index=False, caption="Generalization ability across different partitioners", label="tab:experiment_results").replace("\\begin{table}", "\\begin{table}\n\\begin{center}").replace("\\end{table}", "\\end{center}\n\\end{table}").replace("\\toprule", "\\hline").replace("\\midrule", "\\hline").replace("\\bottomrule", "\\hline").replace("fashion_mnist", "fashion\_mnist")
print(latex_table)

,dataset,train_partitioner,test_partitioner,rmse,r_squared,mae
1,cifar10,iid,dirichlet,0.166415,-3.004208,0.128241
2,cifar10,iid,pathological,0.228688,-1.964022,0.210500
3,cifar10,iid,iid,0.033549,0.642608,0.028195
5,cifar10,dirichlet,iid,0.049900,0.277938,0.044841
6,cifar10,dirichlet,pathological,0.117537,0.217041,0.087326
7,cifar10,dirichlet,dirichlet,0.065471,0.359692,0.052797
9,cifar10,pathological,iid,0.098939,-1.838632,0.092005
10,cifar10,pathological,dirichlet,0.091207,-0.202778,0.076624
11,cifar10,pathological,pathological,0.081866,0.628030,0.069037
13,cifar100,iid,dirichlet,0.046000,0.545147,0.038918


\begin{table}
\begin{center}
\caption{Generalization ability across different partitioners}
\label{tab:experiment_results}
\begin{tabular}{lllrrr}
\hline
dataset & train_partitioner & test_partitioner & rmse & r_squared & mae \\
\hline
cifar10 & iid & dirichlet & 0.166415 & -3.004208 & 0.128241 \\
cifar10 & iid & pathological & 0.228688 & -1.964022 & 0.210500 \\
cifar10 & iid & iid & 0.033549 & 0.642608 & 0.028195 \\
cifar10 & dirichlet & iid & 0.049900 & 0.277938 & 0.044841 \\
cifar10 & dirichlet & pathological & 0.117537 & 0.217041 & 0.087326 \\
cifar10 & dirichlet & dirichlet & 0.065471 & 0.359692 & 0.052797 \\
cifar10 & pathological & iid & 0.098939 & -1.838632 & 0.092005 \\
cifar10 & pathological & dirichlet & 0.091207 & -0.202778 & 0.076624 \\
cifar10 & pathological & pathological & 0.081866 & 0.628030 & 0.069037 \\
cifar100 & iid & dirichlet & 0.046000 & 0.545147 & 0.038918 \\
cifar100 & iid & pathological & 0.035793 & 0.663589 & 0.030645 \\
cifar100 & iid & iid & 0.029988 & 0.7

In [90]:
print(hetero_metric)

earths_mover_distance


In [48]:
display(experiment_results_df[experiment_results_df['set'] == 'train'])

,dataset,train_partitioner,test_partitioner,rmse,r_squared,mae,set
0,cifar10,iid,iid,0.033775,0.676218,0.029098,train
4,cifar10,dirichlet,dirichlet,0.062723,0.434992,0.052070,train
8,cifar10,pathological,pathological,0.082065,0.615140,0.066639,train
12,cifar100,iid,iid,0.030832,0.747810,0.024751,train
16,cifar100,dirichlet,dirichlet,0.044844,0.569671,0.038150,train
20,cifar100,pathological,pathological,0.034037,0.691540,0.028671,train


In [49]:
# Filter the train and test sets
train_df = experiment_results_df[experiment_results_df['set'] == 'train']
test_df = experiment_results_df[experiment_results_df['set'] == 'test']

# Merge the test set with the train set on 'dataset' and 'train_partitioner'
merged_df = test_df.merge(train_df[['dataset', 'train_partitioner', 'rmse', 'r_squared', 'mae']],
                          on=['dataset', 'train_partitioner'],
                          suffixes=('', '_train'))

# Rename the columns to add the 'train_' prefix
merged_df.rename(columns={'rmse_train': 'train_rmse', 'r_squared_train': 'train_r_squared', 'mae_train': 'train_mae'}, inplace=True)

# Display the merged DataFrame
display(merged_df)

,dataset,train_partitioner,test_partitioner,rmse,r_squared,mae,set,train_rmse,train_r_squared,train_mae
0,cifar10,iid,dirichlet,0.166415,-3.004208,0.128241,test,0.033775,0.676218,0.029098
1,cifar10,iid,pathological,0.228688,-1.964022,0.210500,test,0.033775,0.676218,0.029098
2,cifar10,iid,iid,0.033549,0.642608,0.028195,test,0.033775,0.676218,0.029098
3,cifar10,dirichlet,iid,0.049900,0.277938,0.044841,test,0.062723,0.434992,0.052070
4,cifar10,dirichlet,pathological,0.117537,0.217041,0.087326,test,0.062723,0.434992,0.052070
5,cifar10,dirichlet,dirichlet,0.065471,0.359692,0.052797,test,0.062723,0.434992,0.052070
6,cifar10,pathological,iid,0.098939,-1.838632,0.092005,test,0.082065,0.615140,0.066639
7,cifar10,pathological,dirichlet,0.091207,-0.202778,0.076624,test,0.082065,0.615140,0.066639
8,cifar10,pathological,pathological,0.081866,0.628030,0.069037,test,0.082065,0.615140,0.066639
9,cifar100,iid,dirichlet,0.046000,0.545147,0.038918,test,0.030832,0.747810,0.024751


In [ ]:
# 1. Merge the different partitioners res data together (seperately for train and test) for each datasets
# 1.1 Linear regression on this data

# 2. Create the Linear Regression with grouping indicator for pairs and triplets of partitioners

# 3. Create the Linear Regression for each ds, partitioner pair seperately
